In [1]:
!nvidia-smi

Tue Aug 13 08:44:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -qU transformers datasets
!pip install -qU accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-

In [3]:
import pandas as pd
import nltk
import torch

from nltk.tokenize import sent_tokenize
from transformers import (
    pipeline,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    set_seed)
from datasets import load_dataset, load_from_disk, load_metric

from tqdm.auto import tqdm
from matplotlib import pyplot as plt

In [4]:
# gpu check in torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

#### Load Model

In [6]:
model_ckpt = "google/pegasus-cnn_dailymail"

# Load the tokenizer associated with the Pegasus model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, clean_up_tokenization_spaces=True)

# Instantiate the Pegasus model for sequence-to-sequence tasks
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train with pretrained model in custom data

In [7]:
# download and unzip data
!wget -q https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip -q summarizer-data.zip

In [8]:
dataset_loader = load_from_disk('samsum_dataset')
dataset_loader

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
print(f"Features: {dataset_loader['train'].features}")

Features: {'id': Value(dtype='string', id=None), 'dialogue': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None)}


In [10]:
split_rows = [dataset_loader[split].num_rows for split in dataset_loader]
split_rows

[14732, 819, 818]

In [11]:
# sample data
print("Original Dialog::::::")
print(dataset_loader["train"][1]["dialogue"])
print("*"*10)
print("Summary::::::")
print(dataset_loader["train"][1]["summary"])

Original Dialog::::::
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great
**********
Summary::::::
Olivia and Olivier are voting for liberals in this election. 


In [12]:
def convert_to_features(batch_data):
    # Tokenize the dialogue in the batch using the tokenizer
    input_encodings = tokenizer(batch_data['dialogue'], max_length=1024, truncation=True)

    # Tokenize the summary in the batch using the tokenizer as the target tokenizer
    target_encodings = tokenizer(batch_data['summary'], max_length=128, truncation=True, text_target=batch_data['summary'])

    # Return the converted features as a dictionary
    return {
        'input_ids': input_encodings['input_ids'],         # Input token IDs
        'attention_mask': input_encodings['attention_mask'],  # Attention mask
        'labels': target_encodings['input_ids']             # Target token IDs for the model's training
    }



In [13]:
dataset_batch_preprocess = dataset_loader.map(convert_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [14]:
dataset_batch_preprocess

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [15]:
sample_data = dataset_batch_preprocess['train'][1]
print(f"id: {sample_data['id']}")
print("*"*30)
print(f"dialogue length::{len(sample_data['dialogue'].split())} \n{sample_data['dialogue']}")
print("*"*30)
print(f"summary length::{len(sample_data['summary'].split())} \n{sample_data['summary']}")
print("*"*30)
print(f"input_ids length::{len(sample_data['input_ids'])} \n{sample_data['input_ids']}")
print("*"*30)
print(f"attention_mask length::{len(sample_data['attention_mask'])} \n{sample_data['attention_mask']}")
print("*"*30)
print(f"labels length::{len(sample_data['labels'])} \n{sample_data['labels']}")

id: 13728867
******************************
dialogue length::18 
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great
******************************
summary length::10 
Olivia and Olivier are voting for liberals in this election. 
******************************
input_ids length::26 
[18038, 151, 2632, 127, 119, 6228, 118, 115, 136, 2974, 152, 10463, 151, 35884, 130, 329, 107, 18038, 151, 2587, 314, 1242, 10463, 151, 1509, 1]
******************************
attention_mask length::26 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
******************************
labels length::12 
[18038, 111, 34296, 127, 6228, 118, 33195, 115, 136, 2974, 107, 1]


In [17]:
print(f"target decoding: {tokenizer.decode(sample_data['labels'])}")
print(f"input decoding: {tokenizer.decode(sample_data['input_ids'])}")

target decoding: Olivia and Olivier are voting for liberals in this election.</s>
input decoding: Olivia: Who are you voting for in this election? Oliver: Liberals as always. Olivia: Me too!! Oliver: Great</s>


## Evaluating pretrained model

*   `num_beams` refers to beam search, which is used for text generation. It returns the n most probable next words, rather than greedy search which returns the most probable next word.


In [18]:
gen_kwargs = {"length_penalty": 0.7, "num_beams":8, "max_length": 32}
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")

sample_text = dataset_loader["test"][0]["dialogue"]
reference_summary = dataset_loader["test"][0]["summary"]

print("Dialogue:")
print(sample_text)

print('-'*25)
print("\nReference Summary:")
print(reference_summary)

print("\nModel Summary: ")
model_summary = pipe(sample_text, **gen_kwargs)
print(model_summary[0]["summary_text"])

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary: 
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him 


## Training

In [19]:
from transformers import (DataCollatorForSeq2Seq,
                          TrainingArguments,
                          Trainer)

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=5,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.001,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=500,
    save_steps=500,
    gradient_accumulation_steps=16
)

In [20]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

#### **flops**

Confusingly both FLOPs, floating point operations, and FLOPS, floating point operations per second, are used in reference to machine learning. FLOPs are often used to describe how many operations are required to run a single instance of a given model, like VGG19. This is the usage of FLOPs in both of the links you posted, though unfortunately the opengenus link incorrectly mistakenly uses 'Floating point operations per second' to refer to FLOPs.

**Source:** [stack-overflow](https://stackoverflow.com/questions/58498651/what-is-flops-in-field-of-deep-learning)

In [21]:
import warnings
# Suppress the warning about the deprecated AdamW implementation
warnings.filterwarnings("ignore")

trainer = Trainer(
    model=model_pegasus,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_batch_preprocess["test"],
    eval_dataset=dataset_batch_preprocess["validation"]
)

trainer.train()

Step,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


TrainOutput(global_step=255, training_loss=2.2735277774287206, metrics={'train_runtime': 822.4771, 'train_samples_per_second': 4.979, 'train_steps_per_second': 0.31, 'total_flos': 1562486347210752.0, 'train_loss': 2.2735277774287206, 'epoch': 4.981684981684982})

In [22]:
def generate_batch_sized_chunks(list_of_elements, batch_size: int):
    '''
    Generate batch-sized chunks from a given list of elements.

    Args:
        list_of_elements (List): The input list of elements.
        batch_size (int): The desired batch size for chunking.

    Yields:
        List: A batch-sized chunk of elements from the input list.

    Example:
        >>> elements = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        >>> batch_size = 3
        >>> for batch in generate_batch_sized_chunks(elements, batch_size):
        >>>     print(batch)
        [1, 2, 3]
        [4, 5, 6]
        [7, 8, 9]
    '''
    for index in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[index: index + batch_size]

### ROUGE Score

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a suite of metrics used to assess the quality of automatic summaries by comparing them to reference summaries. It is widely employed in natural language processing and text summarization tasks.

#### Mathematical Formulas

1. **ROUGE-N** (n-gram overlap):
   - **Precision (P):** Calculates the proportion of $n$-grams in the generated summary that also appear in the reference summary.
   
     $$
     P_{n} = \frac{\text{Number of overlapping n-grams}}{\text{Total number of n-grams in the generated summary}}
     $$
   - **Recall (R):** Measures the proportion of n-grams in the reference summary that are present in the generated summary.
     $$
     R_{n} = \frac{\text{Number of overlapping n-grams}}{\text{Total number of n-grams in the reference summary}}
     $$
   - **F1-Score:** The harmonic mean of precision and recall.
     $$
     F_{1} = \frac{2 \cdot P_{n} \cdot R_{n}}{P_{n} + R_{n}}
     $$

2. **ROUGE-L** (Longest Common Subsequence):
   - **Precision (P):** Measures the ratio of the length of the longest common subsequence (LCS) between the generated and reference summaries to the length of the generated summary.
     $$
     P_{L} = \frac{\text{Length of LCS}}{\text{Length of the generated summary}}
     $$
   - **Recall (R):** Measures the ratio of the length of the longest common subsequence (LCS) to the length of the reference summary.
     $$
     R_{L} = \frac{\text{Length of LCS}}{\text{Length of the reference summary}}
     $$
   - **F1-Score:** The harmonic mean of precision and recall for ROUGE-L.
     $$
     F_{1L} = \frac{2 \cdot P_{L} \cdot R_{L}}{P_{L} + R_{L}}
     $$

### Explanation

- **ROUGE-N** metrics evaluate the overlap of n-grams (continuous sequences of n words) between the generated and reference summaries. For instance, ROUGE-1 measures unigrams, ROUGE-2 assesses bigrams, and so forth.
  
- **ROUGE-L** assesses the overlap based on the longest common subsequence, which helps to capture the fluency and order of words in the summaries.

In [ ]:
def calculate_metric(dataset,
                     metric,
                     model,
                     tokenizer,
                     batch_size=16,
                     device=device,
                     column_text='articles', column_summary='highlights'):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  target_batches= list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(zip(article_batches, target_batches)):

    inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                    padding="max_length", return_tensors="pt")

    summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                attention_mask=inputs["attention_mask"].to(device),
                                length_penalty=0.8, # parameter for length penalty ensures that the model does not generate sequences that are too long.
                                num_beams=8,
                                max_length=128)

    # Finally, we decode the generated texts,
    # replace the  token, and add the decoded texts with the references to the metric.
    decoded_summaries = [tokenizer.decode(summary, skip_special_tokens=True,
                                          clean_up_tokenization_spaces=True)
                          for summary in summaries]

    decoded_summaries = [summary.replace("", " ") for summary in decoded_summaries]

    metric.add_batch(predictions=decoded_summaries, references=target_batch)

  #  Finally compute and return the ROUGE scores.
  score = metric.compute()
  return score

In [ ]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
from datasets import load_metric

# Use 'load_metric' from 'datasets' to load the 'rouge' metric
rouge_metric = load_metric('rouge')

In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
score = calculate_metric(
    dataset_loader['test'][0:10],
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size = 2,
    column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )


0it [00:00, ?it/s]

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.023683,0.0,0.023547,0.02358


### Save model and tokenizer

In [ ]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json')

#### Load the tokenizer

In [ ]:
# Load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
# Prediction
sample_text = dataset_loader["test"][0]["dialogue"]
reference_summary = dataset_loader["test"][0]["summary"]

print("Dialogue:")
print(sample_text)
print('-'*25)
print("\nReference Summary:")
print(reference_summary)

Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [ ]:
gen_kwargs = {"length_penalty": 0.4, "num_beams":8, "max_length": 32}
pipe = pipeline("summarization", model="pegasus-samsum-model", tokenizer=tokenizer)

print("\nModel Summary:")
model_summary = pipe(sample_text, **gen_kwargs)
print(model_summary[0]["summary_text"])


Model Summary:
Amanda can't find Betty's number. Larry called her last time they were at the park together. She'd rather she text him.
